In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from snowflake.snowpark import Session

In [2]:
load_dotenv()

True

In [3]:
ACCOUNT = os.getenv('ACCOUNT')
USER = os.getenv('SP_USER')
PASSWORD = os.getenv('PASSWORD')
ROLE = os.getenv('ROLE')
WAREHOUSE = os.getenv('WAREHOUSE')
DATABASE = os.getenv('DATABASE')
SCHEMA = os.getenv('SCHEMA')

print(ACCOUNT, USER, ROLE, WAREHOUSE, DATABASE, SCHEMA)

hum.us-east-2.aws ericatuva UVARND UVARND_WH CORE CLIENT


In [4]:
connection_parameters = {
    'account': os.getenv('ACCOUNT'),
    'user': USER,
    'password': os.getenv('PASSWORD'),
    'role': 'RUP',
    #'role': os.getenv('ROLE'),
    'warehouse': os.getenv('WAREHOUSE'),
    'database': os.getenv('DATABASE'),
    'schema': os.getenv('SCHEMA')
}

In [5]:
test_session = Session.builder.configs(connection_parameters).create()

In [6]:
events_query = """
SELECT 
    p.user_id
    , DATE_TRUNC('month', e.day) AS month
    , COUNT(DISTINCT CASE WHEN e.event = 'cite' THEN e.id END) AS cite
    , COUNT(DISTINCT CASE WHEN e.event = 'pageview' THEN e.id END) AS page_view
    , COUNT(DISTINCT CASE WHEN e.event = 'pdf-click' THEN e.id END) AS pdf_click
    , COUNT(DISTINCT CASE WHEN e.event = 'post-read-start' THEN e.id END) AS post_read_start
    , COUNT(DISTINCT CASE WHEN e.event = 'post-read-mid' THEN e.id END) AS post_read_mid
    , COUNT(DISTINCT CASE WHEN e.event = 'post-read-end' THEN e.id END) AS post_read_end
FROM event e
JOIN PROFILE p on e.set_profile = p.id
WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
AND p.user_id IS NOT NULL
GROUP BY 1, 2
ORDER BY 1, 2
"""

In [7]:
print('querying...')
query_results = test_session.sql(events_query).collect()
print('query done')
test_session.close()
print('session closed')

querying...
query done
session closed


In [8]:
# convert to pandas df
query_json = list(map(lambda x: x.as_dict(), query_results))
query_df = pd.DataFrame(query_json)
query_df.to_csv('agg_data.csv', index = 0)
# query_df = pd.read_csv('events_data.csv')

In [9]:
query_df

,USER_ID,MONTH,CITE,PAGE_VIEW,PDF_CLICK,POST_READ_START,POST_READ_MID,POST_READ_END
0,0909pippoyu@sina.com,2022-11-01,0,50,0,6,5,1
1,0909pippoyu@sina.com,2022-12-01,0,9,0,0,0,0
2,0cmrm003@mail.u-tokai.ac.jp,2022-06-01,0,8,0,0,0,0
3,101012573@seu.edu.cn,2022-04-01,0,8,0,0,0,0
4,1043422594@qq.com,2022-10-01,0,4,0,1,1,1
...,...,...,...,...,...,...,...,...
9358,zyang@scripps.edu,2022-04-01,0,23,0,0,0,0
9359,zyang@scripps.edu,2022-05-01,0,39,0,0,0,0
9360,zzhu@partners.org,2022-11-01,0,4,0,1,1,1
9361,zzw1996@jmu.edu.cn,2022-11-01,0,14,0,8,8,5


In [10]:
month_df = pd.DataFrame(query_df.groupby('USER_ID')['MONTH'].apply(list))
cite_df = pd.DataFrame(query_df.groupby('USER_ID')['CITE'].apply(list))
page_view_df = pd.DataFrame(query_df.groupby('USER_ID')['PAGE_VIEW'].apply(list))
pdf_click_df = pd.DataFrame(query_df.groupby('USER_ID')['PDF_CLICK'].apply(list))
post_read_start_df = pd.DataFrame(query_df.groupby('USER_ID')['POST_READ_START'].apply(list))
post_read_mid_df = pd.DataFrame(query_df.groupby('USER_ID')['POST_READ_MID'].apply(list))
post_read_end_df = pd.DataFrame(query_df.groupby('USER_ID')['POST_READ_END'].apply(list))

compressed_df = pd.concat([
    month_df, 
    cite_df, 
    page_view_df,
    pdf_click_df, 
    post_read_start_df, 
    post_read_mid_df, 
    post_read_end_df
], axis = 1)

In [11]:
compressed_df

,MONTH,CITE,PAGE_VIEW,PDF_CLICK,POST_READ_START,POST_READ_MID,POST_READ_END
USER_ID,,,,,,,
0909pippoyu@sina.com,"[2022-11-01, 2022-12-01]","[0, 0]","[50, 9]","[0, 0]","[6, 0]","[5, 0]","[1, 0]"
0cmrm003@mail.u-tokai.ac.jp,[2022-06-01],[0],[8],[0],[0],[0],[0]
101012573@seu.edu.cn,[2022-04-01],[0],[8],[0],[0],[0],[0]
1043422594@qq.com,"[2022-10-01, 2022-11-01]","[0, 0]","[4, 7]","[0, 0]","[1, 3]","[1, 3]","[1, 3]"
1072966956@qq.com,[2022-08-01],[0],[23],[0],[4],[4],[3]
...,...,...,...,...,...,...,...
zy0707zoe@163.com,[2022-09-01],[0],[18],[0],[0],[0],[0]
zyang@scripps.edu,"[2022-04-01, 2022-05-01]","[0, 0]","[23, 39]","[0, 0]","[0, 0]","[0, 0]","[0, 0]"
zzhu@partners.org,[2022-11-01],[0],[4],[0],[1],[1],[1]


In [12]:
compressed_df.to_csv('compressed_agg_data.csv', index = 0)